# League of League of Legends Role Prediction from Post-Game Data

**Name(s)**: Brandon Dioneda, Samuel Zhang

**Website Link**: https://apm-denizens.github.io/dsc80-project5/

## Code

In [29]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

import plotly.express as px
import plotly.graph_objs as go

from IPython.display import display

import black

### Framing the Problem

Predict which role (top-lane, jungle, support, etc.) a player played given their post-game data.

In [4]:
# Load Dataset
league_fp = os.path.join('data', '2022_LoL_esports_match_data_from_OraclesElixir.csv')
league_raw = pd.read_csv(league_fp)

league_raw.dtypes

/Users/elim-mbp-01/.pyenv/versions/3.8.16/envs/dsc80_39/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


gameid               object
datacompleteness     object
url                  object
league               object
year                  int64
                     ...   
assistsat15         float64
deathsat15          float64
opp_killsat15       float64
opp_assistsat15     float64
opp_deathsat15      float64
Length: 123, dtype: object

In [5]:
# =======================
# CLEANING 
# =======================

league = league_raw.copy()

# =======================================================
# Some rows are marked with datacompleteness = "partial"/"ignore".
# According to Costin, this label just means that there's a NaN value in the row (excluding NaN's from team/player related columns)
# Will not ignore.
# league = league[league["datacompleteness"] == "complete"]
# =======================================================

# =======================================================
# Each 'gameid' corresponds to up to 12 rows – one for each of the 5 players on both teams and 2 containing summary data for the two teams (try to find out what distinguishes those rows). After selecting your line of inquiry, make sure to remove either the player rows or the team rows so as not to have issues later in your analysis.

# 24900 rows without a champion 
# (league_raw["champion"].isna()).sum()
# 149400 rows total. 12 rows per match. 149400 / 12 = 12450 matches. 
# 12450 * 2 = 24900 team related rows
# (149400 / 12) = 12450 matches. 12450 * 2 = 24900
# Filtering by whether there's a champion listed or not, should get only the player related rows
league = league[league["champion"].notna()]

# Also drop the team related columns. Every single value in those columns should be null for the player related rows
team_columns_mask = league.isnull().sum(axis=0) == league.shape[0]
league = league.drop(columns=league.columns[team_columns_mask])
print(f"{team_columns_mask.sum()} team-related columns removed")
# =======================================================

# =======================================================
# NaN values for players are represented as "unkown player"
# Mark these as NaN
league["playername"] = league["playername"].apply(lambda val: np.nan if val == "unknown player" else val)
# =======================================================


# =======================================================
# Many columns should be of type bool but are not.
# Checks each column to see if the set of unique values is {0, 1}
# If so, converts the column to bool
columns_to_bool = []
for column in league.columns:
    value_counts = league[column].value_counts()
    if set(value_counts.index) == {0, 1}:
        columns_to_bool.append(column)
        league[column] = league[column].astype(bool)
print(f"CONVERTED {len(columns_to_bool)} COLUMNS TO BOOL")
# =======================================================

# =======================================================
league = league.convert_dtypes()
league.info()
# =======================================================

26 team-related columns removed
CONVERTED 6 COLUMNS TO BOOL
<class 'pandas.core.frame.DataFrame'>
Int64Index: 124500 entries, 0 to 149397
Data columns (total 97 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   gameid                    124500 non-null  string 
 1   datacompleteness          124500 non-null  string 
 2   url                       18680 non-null   string 
 3   league                    124500 non-null  string 
 4   year                      124500 non-null  Int64  
 5   split                     90070 non-null   string 
 6   playoffs                  124500 non-null  boolean
 7   date                      124500 non-null  string 
 8   game                      124500 non-null  Int64  
 9   patch                     124410 non-null  Float64
 10  participantid             124500 non-null  Int64  
 11  side                      124500 non-null  string 
 12  position                  124500 non-nul

In [6]:
league.head()

,gameid,datacompleteness,url,league,year,split,playoffs,date,game,patch,...,opp_csat15,golddiffat15,xpdiffat15,csdiffat15,killsat15,assistsat15,deathsat15,opp_killsat15,opp_assistsat15,opp_deathsat15
0,ESPORTSTMNT01_2690210,complete,<NA>,LCK CL,2022,Spring,False,2022-01-10 07:44:08,1,12.01,...,121,391,345,14,0,1,0,0,1,0
1,ESPORTSTMNT01_2690210,complete,<NA>,LCK CL,2022,Spring,False,2022-01-10 07:44:08,1,12.01,...,100,541,-275,-11,2,3,2,0,5,1
2,ESPORTSTMNT01_2690210,complete,<NA>,LCK CL,2022,Spring,False,2022-01-10 07:44:08,1,12.01,...,119,-475,153,1,0,3,0,3,3,2
3,ESPORTSTMNT01_2690210,complete,<NA>,LCK CL,2022,Spring,False,2022-01-10 07:44:08,1,12.01,...,149,-793,-1343,-34,2,1,2,3,3,0
4,ESPORTSTMNT01_2690210,complete,<NA>,LCK CL,2022,Spring,False,2022-01-10 07:44:08,1,12.01,...,21,443,-497,7,1,2,2,0,6,2


### Generating Train/Val/Test Splits

In [111]:
# Note that class labels are uniform. 
league["position"].value_counts()

top    24900
jng    24900
mid    24900
bot    24900
sup    24900
Name: position, dtype: Int64

In [8]:
league.shape

(124500, 97)

In [9]:
X = league.drop(columns=["position"])
y = league["position"]

X_2, X_test, y_2, y_test = train_test_split(X, y, test_size=0.2, random_state=16)
X_train, X_val, y_train, y_val = train_test_split(X_2, y_2, test_size=0.25, random_state=16) 
# 0.8 * 0.25 = 0.2


In [10]:
X_train.shape

(74700, 96)

### Baseline Model

In [53]:
league["kills"].isna().sum(), league["deaths"].isna().sum()

(0, 0)

In [56]:
((league["kills"] / (league["deaths"]+1)) == float("inf")).sum()

0

In [59]:
preproc_base = ColumnTransformer(
    transformers=[
        ("champion", OneHotEncoder(handle_unknown="ignore"), ["champion"]),
        (
            "kill-death ratio",
            FunctionTransformer(
                lambda df: (df["kills"] / (df["deaths"] + 1)).astype("float32").to_frame()
            ),
            ["kills", "deaths"],
        ),
    ],
    remainder="drop",
)

league_pl_base = Pipeline(
    [
        ("preprocessor", preproc_base),
        ("random-forest", RandomForestClassifier(random_state=16))
    ]
)

In [60]:
# EVALUATION OF BASELINE MODEL
league_pl_base.fit(X_train, y_train)
league_pl_base.score(X_train, y_train), league_pl_base.score(X_val, y_val)

(0.9421552878179384, 0.930843373493976)

In [70]:
pd.Series(league_pl_base.predict(X_train)).value_counts()

bot    15346
top    15135
sup    14938
mid    14666
jng    14615
dtype: int64

In [71]:
y_train.value_counts()

jng    15023
sup    15019
top    14971
bot    14889
mid    14798
Name: position, dtype: Int64

### Final Model

In [ ]:
# GRIDSEARCH

hyperparameters = {
    'random-forestn_estimators': [25, 50, 100, 150],
    'random-forestcriterion': ['gini', 'entropy'],
    'random-forestmax_depth' : [5, 10, 15, None],
    'random-forestmax_features': ['sqrt', 'log2']
}

grid_search = GridSearchCV(league_pl_base, hyperparameters, cv=5)
grid_search.fit(X_train, y_train)
print(grid_search.best_params_)

In [112]:
preproc_final = ColumnTransformer(
    transformers=[
        ("champion", OneHotEncoder(handle_unknown="ignore"), ["champion"]),
        (
            "kill-death ratio",
            FunctionTransformer(
                lambda df: (df["kills"] / df["deaths"])
                .astype("float32")
                .replace(np.inf, np.NaN)
                .fillna(0)
                .to_frame()
            ),
            ["kills", "deaths"],
        ),
        (
            "standardization",
            Pipeline(
                [
                    ("impute", SimpleImputer(strategy="constant", fill_value=0)),
                    ("scale", StandardScaler()),
                ]
            ),
            [
                "cspm",
                "earnedgold",
                "earned gpm",
            ],
        ),
        (
            "quantile-transform",
            Pipeline(
                [
                    ("impute", SimpleImputer(strategy="constant", fill_value=0)),
                    (
                        "quantile",
                        QuantileTransformer(
                            n_quantiles=100, output_distribution="normal"
                        ),
                    ),
                ]
            ),
            ["visionscore", "vspm", "wardsplaced", "wpm",],
        ),
    ],
    remainder="drop",
)

league_pl_final = Pipeline(
    [
        ("preprocessor", preproc_final),
        (
            "random-forest",
            RandomForestClassifier(
                # BEST COMBINATION OF HYPERPARAMETERS ACCORDING TO GRIDSEARCH
                random_state=16,
                criterion="gini",
                max_depth=None,
                max_features="log2",
                n_estimators=150,
            ),
        ),
    ]
)


In [95]:
hist_trace = go.Histogram(x=league["visionscore"], nbinsx=25)
layout = go.Layout(title="Visionscore Histogram", xaxis_title="Visionscore", yaxis_title="Frequency")
fig = go.Figure(data=[hist_trace], layout=layout)
# fig.write_html("visionscore_histogram.html")
fig.show()

In [96]:
hist_trace = go.Histogram(x=league["earnedgold"], nbinsx=25)
layout = go.Layout(title="Earned Gold Histogram", xaxis_title="Earned Gold", yaxis_title="Frequency")
fig = go.Figure(data=[hist_trace], layout=layout)
# fig.write_html("earnedgold_histogram.html")
fig.show()

In [32]:

for colname in [
                "visionscore",
                "earned gpm",
                "cspm",
                "vspm",
                "earnedgold",
                "wardsplaced",
                "wpm",
            ]:
    print(f"{colname}: {league[colname].isna().sum()}")

visionscore: 10
earned gpm: 0
cspm: 0
vspm: 10
earnedgold: 0
wardsplaced: 10
wpm: 10


In [33]:
# EVALUATION OF FINAL MODEL TRAIN & VALIDATION SETS
league_pl_final.fit(X_train, y_train)
league_pl_final.score(X_train, y_train), league_pl_final.score(X_val, y_val)

(1.0, 0.9572690763052208)

In [70]:
# EVALUATION OF FINAL MODEL TEST SET
league_pl_final.score(X_test, y_test)

0.9568674698795181

### Fairness Analysis

Null Hypothesis: Our model is fair. It's accuracy for the Red & Blue sides are roughly the same, and any differences are due to random chance.  
Alt Hypothesis: Our model is unfair. It's accuracy for the Blue team is higher. 

In [71]:
blue_wr = league.loc[league["side"] == "Blue", "result"].mean()
red_wr = league.loc[league["side"] == "Red", "result"].mean()
print(blue_wr, red_wr, blue_wr + red_wr)

0.5242570281124498 0.47558232931726907 0.9998393574297189


In [72]:
def get_test_stat(league_df: pd.DataFrame):
    league_df_red = league_df.loc[league_df["side"] == "Red"]
    red_score = league_pl_final.score(
        league_df_red.drop(columns=["position"]), league_df_red["position"]
    )

    league_df_blue = league_df.loc[league_df["side"] == "Blue"]
    blue_score = league_pl_final.score(
        league_df_blue.drop(columns=["position"]), league_df_blue["position"]
    )

    return blue_score - red_score

observed_test_stat = get_test_stat(league)
observed_test_stat

0.0008353413654619279

In [22]:
league_cp = league.copy()

test_stats = []
for _ in range(100):
    league_cp["side"] = np.random.permutation(league_cp["side"])
    test_stats.append(get_test_stat(league_cp))
    

In [23]:
test_stats_arr = np.array(test_stats)

In [73]:
(test_stats_arr >= observed_test_stat).mean()

0.13

In [74]:
test_stats_arr

array([-1.06024096e-03,  2.57028112e-04, -4.17670683e-04, -9.63855422e-05,
       -8.99598394e-04,  7.06827309e-04,  3.53413655e-04, -2.89156627e-04,
       -7.38955823e-04,  6.42570281e-04, -7.06827309e-04,  2.57028112e-04,
       -1.28514056e-03,  5.78313253e-04, -8.03212851e-04,  5.46184739e-04,
        9.31726908e-04,  4.49799197e-04,  5.14056225e-04,  3.21285141e-05,
       -7.38955823e-04, -2.57028112e-04, -8.99598394e-04, -1.28514056e-04,
       -7.38955823e-04,  2.57028112e-04,  2.57028112e-04,  1.41365462e-03,
        8.03212851e-04,  8.35341365e-04,  3.21285141e-04,  3.85542169e-04,
        7.06827309e-04, -3.53413655e-04,  1.60642570e-04,  1.28514056e-04,
        1.06024096e-03,  4.81927711e-04,  2.24899598e-04, -1.02811245e-03,
        4.17670683e-04,  1.22088353e-03, -3.21285141e-05, -7.71084337e-04,
       -1.92771084e-04, -1.63855422e-03,  1.28514056e-03, -3.21285141e-04,
        9.95983936e-04,  2.57028112e-04,  1.09236948e-03, -1.25301205e-03,
       -9.63855422e-04,  

In [110]:
hist_trace = go.Histogram(x=test_stats, nbinsx=10)
layout = go.Layout(title="Histogram of Test Statistics", xaxis_title="Test Statistic (Difference in Accuracy)", yaxis_title="Frequency")

fig = go.Figure(data=[hist_trace], layout=layout)
fig.add_vline(x=observed_test_stat, line_width=2, line_dash="solid", line_color="red")

fig.write_html("teststats_histogram.html")
fig.show()